In [ ]:
##
# @file This file is part of EDGE.
#
# @author Alexander Breuer (anbreuer AT ucsd.edu)
#
# @section LICENSE
# Copyright (c) 2016, Regents of the University of California
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# @section DESCRIPTION
# Derivation of basis functions and the attached matrices for triangular elements.
##

In [ ]:
import sympy
import matplotlib.pyplot
sympy.init_printing(use_unicode=True)

In [ ]:
# upper polynomial degree
l_degUp = 2

# basis
l_basis = []

xi1 = sympy.symbols('xi_1') # first triangular coords
xi2 = sympy.symbols('xi_2') # second triangular coord

eta1 = sympy.symbols('eta_1') # first collapsed coord
eta2 = sympy.symbols('eta_2') # scond collapsed coord

# derive basis functions
for l_order in xrange(0, l_degUp+1):
  for l_p in xrange(0, l_order+1):
    # first principal function
    l_psiA = sympy.jacobi( l_p, 0, 0, eta1 )
    
    for l_q in xrange(0, l_order+1):
      if( l_p + l_q == l_order  ): # build hierarchical basis following pascals triangle
        # second principal function
        l_psiB = ( ( (1-eta2)/2 )**l_p * sympy.jacobi( l_q, 2*l_p+1, 0, eta2 ) )

        l_basis = l_basis + [l_psiA * l_psiB]
        
        # insert triangular coords
        l_basis[-1] = l_basis[-1].subs( eta1, 2*(1+xi1)/(1-xi2) - 1 ).subs(eta2, xi2)
        
        # use triangle with xi1 in [0,1], xi2 in [0,1-xi1]
        l_basis[-1] = l_basis[-1].subs(xi1, 2*xi1-1).subs(xi2, 2*xi2 - 1)
        
        # simplify the basis
        l_basis[-1] = sympy.simplify( l_basis[-1] )

In [ ]:
# generate code for the basis
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  val =", sympy.ccode( l_basis[l_b]  ).replace("pow", "std::pow"), ";"
  print "}"

# generate code for the derivatives
print '*******************************'
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  valDxi1 =", sympy.ccode( sympy.diff(l_basis[l_b], xi1)  ).replace("pow", "std::pow"), ";"
  print "}"
print '*******************************'
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  valDxi2 =", sympy.ccode( sympy.diff(l_basis[l_b], xi2)  ).replace("pow", "std::pow"), ";"
  print "}"

In [ ]:
# triangular integration
l_fInt = lambda bf: sympy.integrate( bf, (xi1, 0, 1-xi2), (xi2, 0, 1) )
l_diffXi1 = lambda bf: sympy.diff( bf, xi1)
l_diffXi2 = lambda bf: sympy.diff( bf, xi2)


# compute mass matrix
l_mm = sympy.Matrix( l_basis ) * sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_mm = l_mm.applyfunc( l_fInt)

# compute stiffness matrices
l_sm1 = sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_sm1 = l_sm1.applyfunc( l_diffXi1 )
l_sm1 = sympy.Matrix( l_basis ) * l_sm1
l_sm1 = l_sm1.applyfunc( l_fInt )

l_sm2 = sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_sm2 = l_sm2.applyfunc( l_diffXi2 )
l_sm2 = sympy.Matrix( l_basis ) * l_sm2
l_sm2 = l_sm2.applyfunc( l_fInt )

In [ ]:
# compute flux matrices
l_localFm = dict()
l_neighFm = dict()

chi = sympy.symbols('chi')

# local face
for l_f1 in xrange(0,3):
  # determine basis
  l_basisF1 = []
  for l_bf in xrange(0, len(l_basis)):
    if   l_f1 == 0: l_basisF1 = l_basisF1 + [ l_basis[l_bf].subs(xi1, chi   ).subs( xi2, 0)     ]
    elif l_f1 == 1: l_basisF1 = l_basisF1 + [ l_basis[l_bf].subs(xi1, 1-chi ).subs( xi2, chi)   ]
    elif l_f1 == 2: l_basisF1 = l_basisF1 + [ l_basis[l_bf].subs(xi1, 0     ).subs( xi2, 1-chi) ]
        
  # assemnle flux matrix
  l_localFm['fm'+str(l_f1)] = sympy.Matrix( l_basisF1 ) * \
                              sympy.Matrix.transpose( sympy.Matrix( l_basisF1 ))
    
  # do the integration
  l_localFm['fm'+str(l_f1)] = sympy.integrate( l_localFm['fm'+str(l_f1)], (chi, 0, 1) )
    
  # neighboring face
  for l_f2 in xrange(0,3):
    # basis at face, where chi is the shared integration parameter
    l_basisF2 = []
    
    # iterate over basis functions
    for l_bf in xrange(0, len(l_basis)):
      if   l_f2 == 0: l_basisF2 = l_basisF2 + [ l_basis[l_bf].subs(xi1, 1-chi   ).subs( xi2, 0)   ]
      elif l_f2 == 1: l_basisF2 = l_basisF2 + [ l_basis[l_bf].subs(xi1, chi   ).subs( xi2, 1-chi) ]
      elif l_f2 == 2: l_basisF2 = l_basisF2 + [ l_basis[l_bf].subs(xi1, 0     ).subs( xi2, chi)   ]

    # assemble the flux matrix
    l_neighFm['fp'+str(l_f1)+str(l_f2)] = sympy.Matrix( l_basisF2 ) * \
                                          sympy.Matrix.transpose( sympy.Matrix( l_basisF1 ))

    # do the integration
    l_neighFm['fp'+str(l_f1)+str(l_f2)] = sympy.integrate( l_neighFm['fp'+str(l_f1)+str(l_f2)], (chi, 0, 1) )

In [ ]:
l_mm,l_sm1, l_sm2, l_localFm, l_neighFm